In [ ]:
import numpy as np
import h5py
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Path to your HDF5 file
hdf5_file = "/content/drive/MyDrive/zindi_homesettelment/train_data.h5"

# Open the HDF5 file
with h5py.File(hdf5_file, 'r') as hdf:
    # Extract the images (X)
    X = np.array(hdf['images'])

    # Extract the labels (y)
    y = np.array(hdf['labels'])

# Check the shapes to ensure they are correct
print("Shape of X (images):", X.shape)
print("Shape of y (labels):", y.shape)

hdf5_file_test = "/content/drive/MyDrive/zindi_homesettelment/test_data.h5"
# Open the HDF5 file
with h5py.File(hdf5_file_test, 'r') as hdf:
    # Extract the images (X)
    X_test = np.array(hdf['images'])

# Check the shapes to ensure they are correct
print("Shape of X_test (images):", X_test.shape)

Shape of X (images): (1100000, 16, 16, 6)
Shape of y (labels): (1100000,)
Shape of X_test (images): (120000, 16, 16, 6)


In [ ]:
# Step 1: Count the number of 1's in y
num_ones = np.sum(y == 1)

# Step 2: Get indices of 0's and 1's in y
ones_indices = np.where(y == 1)[0]
zeros_indices = np.where(y == 0)[0]

# Step 3: Randomly sample the same number of 0's as there are 1's
balanced_zero_indices = np.random.choice(zeros_indices, num_ones, replace=False)

# Step 4: Combine indices of 0's and 1's
balanced_indices = np.concatenate([ones_indices, balanced_zero_indices])

# Step 5: Create balanced X and y
X_balanced = X[balanced_indices]
y_balanced = y[balanced_indices]

# Display the number of 0's and 1's in the balanced y
print(f"Number of 1's in balanced y: {np.sum(y_balanced == 1)}")
print(f"Number of 0's in balanced y: {np.sum(y_balanced == 0)}")

Number of 1's in balanced y: 100000
Number of 0's in balanced y: 100000


In [ ]:
del hdf5_file ,hdf5_file_test ,X,y

In [ ]:
from sklearn.utils import shuffle
# Shuffle both X_balanced and y_balanced together
X_train, y_train = shuffle(X_balanced, y_balanced, random_state=1)

In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.metrics import roc_auc_score

In [ ]:
pp = {'max_depth': 13, 'subsample': 0.9943547669638271, 'colsample_bytree': 0.5470176646112033,
                           'gamma': 0.4458781483943156, 'min_child_weight': 10,
                           'reg_alpha': 0.6780947443430723, 'reg_lambda': 0.004616119552006354}

In [ ]:
 {'max_depth': 7, 'learning_rate': 0.08373893897405209, 'n_estimators': 781, 'subsample': 0.593266351710131, 'colsample_bytree': 0.8366435593730454, 'gamma': 2.1832705370225813, 'min_child_weight': 1, 'reg_alpha': 0.7623574051817725, 'reg_lambda': 0.22116499184077165}

In [ ]:
params = {
    'max_depth': 7,
    'subsample': 0.593266351710131,
    'colsample_bytree': 0.8366435593730454,
    'gamma': 2.1832705370225813,
    'min_child_weight': 1,
    'reg_alpha': 0.7623574051817725,
    'reg_lambda': 0.22116499184077165,
    'learning_rate':  0.08373893897405209,
    'n_estimators': 781,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'early_stopping_rounds': 300,
    'random_state': 42,
    'verbose' : 20,
     'tree_method': 'gpu_hist',  # Enable GPU acceleration
}

In [ ]:
params = {
    'max_depth': 7,
    'subsample': 0.593266351710131,
    'colsample_bytree': 0.8366435593730454,
    'gamma': 2.1832705370225813,
    'min_child_weight': 1,
    'reg_alpha': 0.7623574051817725,
    'reg_lambda': 0.22116499184077165,
    'learning_rate':  0.08373893897405209,
    'n_estimators': 2000,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'early_stopping_rounds': 300,
    'random_state': 42,
    'verbose' : 20,
     'tree_method': 'gpu_hist',  # Enable GPU acceleration
}

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
import os

# Create unique IDs for each row in the training and test sets
train_ids = np.arange(X_train.shape[0])  # IDs from 0 to n_train-1
test_ids = np.arange(X_test.shape[0])    # IDs from 0 to n_test-1

test_predictions = []

# Ensure the feature matrices use float32 to reduce memory usage
X_train_flat = X_train.reshape(X_train.shape[0], -1).astype(np.float32)
X_test_flat = X_test.reshape(X_test.shape[0], -1).astype(np.float32)

# Cross-validation setup
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
roc_scores = []

# Directory to save the datasets (optional)
output_dir = 'cv_folds_data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# XGBoost parameters (adjust as needed)


# Use for-loop to perform cross-validation
for fold_idx, (train_idx, valid_idx) in enumerate(skf.split(X_train_flat, y_train)):
    X_train_cv, X_valid_cv = X_train_flat[train_idx], X_train_flat[valid_idx]
    y_train_cv, y_valid_cv = y_train[train_idx], y_train[valid_idx]
    valid_ids_cv = train_ids[valid_idx]  # Corresponding IDs for validation set

    # Create and train XGBoost model
    model = XGBClassifier(**params)
    model.fit(
        X_train_cv, y_train_cv,
        eval_set=[(X_train_cv, y_train_cv), (X_valid_cv, y_valid_cv)],
        verbose=20
    )

    # Predict probabilities for ROC-AUC
    y_valid_pred = model.predict_proba(X_valid_cv)[:, 1]
    del X_valid_cv  # Free up memory

    # Calculate ROC-AUC for this fold
    roc_score = roc_auc_score(y_valid_cv, y_valid_pred)
    roc_scores.append(roc_score)
    print(f"Fold {fold_idx + 1} ROC-AUC score: {roc_score:.4f}")

    # Save the validation set with ids for this fold
    valid_df = pd.DataFrame({
        'id': valid_ids_cv,
        'target': y_valid_cv,
        'predicted_prob': y_valid_pred
    })

    valid_df.to_csv(f"{output_dir}/valid_fold_{fold_idx + 1}.csv", index=False)
    print(f"Saved validation data for fold {fold_idx + 1}")

    del y_valid_cv  # Free up memory

    # Test predictions
    y_test_pred_proba = model.predict_proba(X_test_flat)[:, 1]
    test_predictions.append(y_test_pred_proba)

# After all folds, save the test predictions along with test IDs
test_predictions_mean = np.mean(test_predictions, axis=0)
test_df = pd.DataFrame({
    'id': test_ids,
    'predicted_prob': test_predictions_mean
})
test_df.to_csv(f"{output_dir}/test_predictions.csv", index=False)

print("Test predictions saved.")

# Print average ROC-AUC score across all folds
print(f"Average ROC-AUC score: {np.mean(roc_scores):.4f}")

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:28:03] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:28:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.72938	validation_1-auc:0.71142
[20]	validation_0-auc:0.81297	validation_1-auc:0.78244
[40]	validation_0-auc:0.84135	validation_1-auc:0.80361
[60]	validation_0-auc:0.86036	validation_1-auc:0.81627
[80]	validation_0-auc:0.87535	validation_1-auc:0.82570
[100]	validation_0-auc:0.88747	validation_1-auc:0.83204
[120]	validation_0-auc:0.89849	validation_1-auc:0.83752
[140]	validation_0-auc:0.90731	validation_1-auc:0.84176
[160]	validation_0-auc:0.91594	validation_1-auc:0.84595
[180]	validation_0-auc:0.92326	validation_1-auc:0.84937
[200]	validation_0-auc:0.92932	validation_1-auc:0.85164
[220]	validation_0-auc:0.93494	validation_1-auc:0.85390
[240]	validation_0-auc:0.93986	validation_1-auc:0.85549
[260]	validation_0-auc:0.94423	validation_1-auc:0.85740
[280]	validation_0-auc:0.94879	validation_1-auc:0.85898
[300]	validation_0-auc:0.95287	validation_1-auc:0.86023
[320]	validation_0-auc:0.95623	validation_1-auc:0.86120
[340]	validation_0-auc:0.95984	validation_1-auc:0.8624

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:29:23] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:29:23] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


Fold 1 ROC-AUC score: 0.8720
Saved validation data for fold 1


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:30:11] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:30:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


...

Fold 4 ROC-AUC score: 0.8676
Saved validation data for fold 4


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:36:41] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:36:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.73119	validation_1-auc:0.71312
[20]	validation_0-auc:0.81303	validation_1-auc:0.78524
[40]	validation_0-auc:0.84123	validation_1-auc:0.80473
[60]	validation_0-auc:0.85998	validation_1-auc:0.81670
[80]	validation_0-auc:0.87477	validation_1-auc:0.82620
[100]	validation_0-auc:0.88744	validation_1-auc:0.83292
[120]	validation_0-auc:0.89842	validation_1-auc:0.83855
[140]	validation_0-auc:0.90742	validation_1-auc:0.84307
[160]	validation_0-auc:0.91572	validation_1-auc:0.84676
[180]	validation_0-auc:0.92270	validation_1-auc:0.84962
[200]	validation_0-auc:0.92982	validation_1-auc:0.85239
[220]	validation_0-auc:0.93617	validation_1-auc:0.85504
[240]	validation_0-auc:0.94131	validation_1-auc:0.85704
[260]	validation_0-auc:0.94578	validation_1-auc:0.85880
[280]	validation_0-auc:0.94982	validation_1-auc:0.86010
[300]	validation_0-auc:0.95377	validation_1-auc:0.86161
[320]	validation_0-auc:0.95704	validation_1-auc:0.86262
[340]	validation_0-auc:0.96004	validation_1-auc:0.8634

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:38:04] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Fold 5 ROC-AUC score: 0.8729
Saved validation data for fold 5
Test predictions saved.
Average ROC-AUC score: 0.8716


In [ ]:
mean_test_predictions = np.mean(test_predictions, axis=0)
ss = pd.read_csv('/content/drive/MyDrive/zindi_homesettelment/SampleSubmission.csv')
ss['class'] = mean_test_predictions

In [ ]:
gg = pd.read_csv('/content/drive/MyDrive/zindi_homesettelment/id_map.csv')
gg['pred'] = mean_test_predictions
gg

,id,ID,pred
0,id_xjoh8datmz,0,0.443913
1,id_ui7ciy2r2l,1,0.443668
2,id_le7254d4ch,2,0.477274
3,id_99qf1p6vg0,3,0.083336
4,id_jc5kg2grpc,4,0.393910
...,...,...,...
119995,id_fy1pnuvfnv,119995,0.843412
119996,id_ujwqytrncm,119996,0.120257
119997,id_kxz28n3qfv,119997,0.692727
119998,id_yg2enc9wf4,119998,0.051351


In [ ]:
ss = pd.read_csv('/content/drive/MyDrive/zindi_homesettelment/SampleSubmission.csv')
ss_d = pd.merge(ss,gg,on=['id'])
ss_d
ss_d['class'] = ss_d['pred']
ss_d
ss_d = ss_d[['id','class']]
ss_d.to_csv('xgb_tuned_new.csv',index=False)

In [ ]:
df1 = pd.read_csv('/content/cv_folds_data/valid_fold_1.csv')
df2 = pd.read_csv('/content/cv_folds_data/valid_fold_2.csv')
df3 =  pd.read_csv('/content/cv_folds_data/valid_fold_3.csv')
df4 =  pd.read_csv('/content/cv_folds_data/valid_fold_4.csv')
df5 =  pd.read_csv('/content/cv_folds_data/valid_fold_5.csv')

In [ ]:
# Combine DataFrames
combined_df = pd.concat([df1, df2, df3,df4,df5], ignore_index=True)

print(combined_df)


            id  target  predicted_prob
0            3       1        0.484352
1            5       0        0.426525
2           16       0        0.004413
3           27       1        0.381051
4           31       0        0.053527
...        ...     ...             ...
199995  199980       1        0.634755
199996  199986       1        0.562514
199997  199989       0        0.095566
199998  199994       0        0.216264
199999  199996       0        0.048291

[200000 rows x 3 columns]


In [ ]:
combined_df.to_csv('xgb_tuned_full_new.csv',index=False)

# optuna

In [ ]:
!pip install optuna

In [ ]:
X_train_flat = X_train.reshape(X_train.shape[0], -1).astype(np.float32)
X_test_flat = X_test.reshape(X_test.shape[0], -1).astype(np.float32)
del X_train, X_test

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_flat, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
import os

In [ ]:
import optuna
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# Define the objective function for Optuna to minimize
def objective(trial):
    # Suggest hyperparameters to tune
    params = {
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'random_state': 42,
        'early_stopping_rounds':200,
        'tree_method': 'gpu_hist',  # Use GPU if available
        'objective': 'binary:logistic',  # Modify as per your task (binary/multiclass)
        'eval_metric': 'auc',
    }

    # Train the model with the suggested hyperparameters
    model = XGBClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        verbose=20
    )

    # Predict on the validation set
    y_valid_pred = model.predict_proba(X_valid)[:, 1]

    # Calculate and return the ROC-AUC score
    roc_score = roc_auc_score(y_valid, y_valid_pred)
    return roc_score

# Create the study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Print the best hyperparameters
print(f"Best hyperparameters: {study.best_params}")

# Use the best hyperparameters to train the final model
best_model = XGBClassifier(**study.best_params)
best_model.fit(X_train, y_train)

# Predict on the validation set using the best model
y_valid_best_pred = best_model.predict_proba(X_valid)[:, 1]
roc_auc_best = roc_auc_score(y_valid, y_valid_best_pred)

print(f"Best ROC-AUC score on validation data: {roc_auc_best:.4f}")


[I 2024-09-17 08:20:32,972] A new study created in memory with name: no-name-e0c161fd-f85a-42e6-bd58-a5706fd107d1
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [08:21:29] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[0]	validation_0-auc:0.72554
[20]	validation_0-auc:0.80606
[40]	validation_0-auc:0.82366
[60]	validation_0-auc:0.83611
[80]	validation_0-auc:0.84308
[100]	validation_0-auc:0.84872
[120]	validation_0-auc:0.85252
[140]	validation_0-auc:0.85584
[160]	validation_0-auc:0.85781
[180]	validation_0-auc:0.85970
[200]	validation_0-auc:0.86102
[220]	validation_0-auc:0.86254
[240]	validation_0-auc:0.86377
[260]	validation_0-auc:0.86492
[280]	validation_0-auc:0.86544
[300]	validation_0-auc:0.86628
[320]	validation_0-auc:0.86664
[340]	validation_0-auc:0.86712
[360]	validation_0-auc:0.86749
[380]	validation_0-auc:0.86758
[400]	validation_0-auc:0.86785
[417]	validation_0-auc:0.86799


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [08:22:30] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [08:22:30] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
[I 2024-09-17 08:22:31,528] Trial 0 finished with value: 0.86799352875 and parameters: {'max_depth': 9, 'lea

[0]	validation_0-auc:0.71861
[20]	validation_0-auc:0.81975
[40]	validation_0-auc:0.83140
[60]	validation_0-auc:0.83629
[80]	validation_0-auc:0.83954
[100]	validation_0-auc:0.84135
[120]	validation_0-auc:0.84278
[140]	validation_0-auc:0.84444
[160]	validation_0-auc:0.84598
[180]	validation_0-auc:0.84680
[200]	validation_0-auc:0.84778
[220]	validation_0-auc:0.84837
[240]	validation_0-auc:0.84934
[260]	validation_0-auc:0.85020
[280]	validation_0-auc:0.85069
[295]	validation_0-auc:0.85131


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [08:24:24] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
[I 2024-09-17 08:24:25,605] Trial 1 finished with value: 0.8513089274999999 and parameters: {'max_depth': 14, 'learning_rate': 0.22609828116379907, 'n_estimators': 296, 'subsample': 0.5278563197231405, 'colsample_bytree': 0.524593667041495, 'gamma': 1.0058414424982627, 'min_child_weight': 7, 'reg_alpha': 0.16763031764924818, 'reg_lambda': 0.9702490076018442}. Best is trial 0 with value: 0.86799352875.
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [08:25:07] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.

...

[0]	validation_0-auc:0.71616
[20]	validation_0-auc:0.80087
[40]	validation_0-auc:0.82019
[60]	validation_0-auc:0.83115
[80]	validation_0-auc:0.83766
[100]	validation_0-auc:0.84273
[120]	validation_0-auc:0.84700
[140]	validation_0-auc:0.85007
[160]	validation_0-auc:0.85194
[180]	validation_0-auc:0.85356
[200]	validation_0-auc:0.85500
[220]	validation_0-auc:0.85621
[240]	validation_0-auc:0.85734
[260]	validation_0-auc:0.85856
[280]	validation_0-auc:0.85949
[300]	validation_0-auc:0.86022
[320]	validation_0-auc:0.86076
[340]	validation_0-auc:0.86119
[360]	validation_0-auc:0.86163
[380]	validation_0-auc:0.86207
[400]	validation_0-auc:0.86242
[420]	validation_0-auc:0.86235
[440]	validation_0-auc:0.86276
[460]	validation_0-auc:0.86312
[480]	validation_0-auc:0.86347
[500]	validation_0-auc:0.86343
[520]	validation_0-auc:0.86368
[540]	validation_0-auc:0.86372
[560]	validation_0-auc:0.86367
[580]	validation_0-auc:0.86364
[600]	validation_0-auc:0.86375
[620]	validation_0-auc:0.86387
[640]	validati

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:17:24] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
[I 2024-09-17 09:17:25,656] Trial 26 finished with value: 0.86479197625 and parameters: {'max_depth': 8, 'learning_rate': 0.1287161788978956, 'n_estimators': 917, 'subsample': 0.5777943760237225, 'colsample_bytree': 0.9660367111980218, 'gamma': 3.090582030400469, 'min_child_weight': 3, 'reg_alpha': 0.006286771140462277, 'reg_lambda': 0.09373043010680729}. Best is trial 14 with value: 0.8703906399999999.
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [09:18:06] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    

[0]	validation_0-auc:0.72162
[20]	validation_0-auc:0.80066
[40]	validation_0-auc:0.80688
[60]	validation_0-auc:0.81080
[80]	validation_0-auc:0.81414
[100]	validation_0-auc:0.81741
[120]	validation_0-auc:0.82055
[140]	validation_0-auc:0.82362
[160]	validation_0-auc:0.82627
[180]	validation_0-auc:0.82877


[W 2024-09-17 09:20:33,114] Trial 27 failed with parameters: {'max_depth': 12, 'learning_rate': 0.011066786043720855, 'n_estimators': 665, 'subsample': 0.6412360333425637, 'colsample_bytree': 0.8904718506286942, 'gamma': 1.82988898424102, 'min_child_weight': 6, 'reg_alpha': 0.9033212507002057, 'reg_lambda': 0.41509843042081185} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-13-e8b2f2d260e8>", line 27, in objective
    model.fit(
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py", line 1531, in fit
    self._Booster = train(
  File "/usr/local/lib/python3.10/dist-packages/xgboost/core.py", line 726, in inner_f
    return func(**kwargs)
  File "/usr/local/lib/python

KeyboardInterrupt: 

In [ ]:
 {'max_depth': 7, 'learning_rate': 0.08373893897405209, 'n_estimators': 781, 'subsample': 0.593266351710131, 'colsample_bytree': 0.8366435593730454, 'gamma': 2.1832705370225813, 'min_child_weight': 1, 'reg_alpha': 0.7623574051817725, 'reg_lambda': 0.22116499184077165}